In [1]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np
import re

In [ ]:
df = pd.read_csv("KG_translation_triplets_results.csv")

In [ ]:
df.dropna(inplace = True)

In [ ]:
df

In [ ]:
def remove(x):
    x = x.replace('"', ' ').lower()
    return re.sub('\W+','_', x)

In [ ]:
df.loc[(df['pred'].str.startswith('=')), 'pred'] = 'equal_to'
df['pred'] = df['pred'].apply(remove)

In [ ]:
def remove(x):
    x = x.replace('"', ' ').lower()
    return x

In [ ]:
df['sub'] = df['sub'].apply(remove)
df['obj'] = df['obj'].apply(remove)

In [ ]:
df

In [ ]:
data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "AMMAAR4ahmad"))
session = data_base_connection.session()   

In [ ]:
def execute_transactions(execution_list): 
    for i in execution_list:
        session.run(i)

In [ ]:
df = df[['info-unit', 'sub', 'pred', 'obj', 'type']]
df = df.drop_duplicates().reset_index(drop = True)

In [ ]:
info_unit = df['info-unit'].unique()
sub_obj = pd.concat([df['sub'], df['obj']]).unique()
print(len(sub_obj))

In [ ]:
# Information Units Nodes
info_nodes = []
for i, info in enumerate(info_unit):
    info_nodes.append('create (t:info_unit {info_unit : "' + info + '"})')
execute_transactions(info_nodes)

In [ ]:
# Subject/Object Nodes
subject_nodes = []
for i, sub in enumerate(sub_obj):
    subject_nodes.append('create (t:subject_object {sub_obj: "' + sub + '"})')
execute_transactions(subject_nodes)

In [ ]:
df

In [ ]:
def startwithdigit(x):
    return x[0].isdigit()
df['valid'] = df['pred'].apply(startwithdigit)
#df = df[df['valid'] == 0].reset_index(drop = True)
#df

In [ ]:
df_temp = df[(df['type'].isin(['C', 'D']))].reset_index(drop = True)
df_temp[df_temp['valid'] == 1]

In [ ]:
#def startwithdigit(x):
#    return x[0].isdigit()
#df['valid'] = df['pred'].apply(startwithdigit)
df[df['valid'] == 1]

In [ ]:
# Triplets C-D
triplet_relation = []
df_temp = df[(df['type'].isin(['C', 'D']))].reset_index(drop = True)
error_index = []
for i in range(len(df_temp)):
    info = df.iloc[i,0]
    pred = df.iloc[i,2]
    obj = df.iloc[i,3]
    print(i, info, pred, obj)
    if(pred[0].isdigit()):
        continue
    triplet_relation.append(f'''MATCH (a:info_unit), (b:subject_object) 
                    WHERE a.info_unit = "{info}" AND b.sub_obj = "{obj}"
                    create (a) -[r:{pred}]-> (b)''')
execute_transactions(triplet_relation)
print(error_index)

In [ ]:
# Triplets A,B and E
triplets = []
df_temp = df[(df['type'].isin(['A', 'B', 'E']))].reset_index(drop = True)
error_index = []
for i in range(len(df)):
    sub = df.iloc[i,1]
    pred = df.iloc[i,2]
    obj = df.iloc[i,3]
    print(i, sub, pred, obj)
    if(pred[0].isdigit()):
        continue
    triplets.append(f'''MATCH (a:subject_object), (b:subject_object) 
                    WHERE a.sub_obj = "{sub}" AND b.sub_obj = "{obj}"
                    create (a) -[r: {pred}]-> (b)''')
execute_transactions(triplets)
print(error_index)

In [ ]:
def research_code(topic):
    research_connect = []
    for i in range(len(df_research)):
        paper = str(df_research.loc[i, 'paper_ID'])
        obj = df_research.loc[i, 'obj']
        research_connect.append(f'''MATCH (a:paper_id), (b:subject_object) 
                                WHERE a.paper_id = "{paper}" AND b.info_unit = "{obj}"
                                create (a) -[r:has_research_problem]-> (b)''')
    execute_transactions(research_connect)

In [ ]:
def paper_graph(topic, paper_id):
    
    df_temp = df[(df['topic'] == topic) & (df['paper_ID'] == paper_id)]
    df_temp = df_temp[['info-unit', 'sub', 'pred', 'obj']]
    
    paper = str(paper_id)
    
    info_unit = df_temp['info-unit'].unique()
    sub_obj = pd.concat([df_temp['sub'], df_temp['obj']]).unique()
    
    
    # Information Units and Paper Relation
    info_connect = []
    for i, info in enumerate(info_unit):
        info_connect.append(f'''MATCH (a:paper_id), (b:info_unit) 
                            WHERE a.paper_id = "{paper}" AND b.info_unit = "{info}"
                            create (a) -[r:has]-> (b)''')
    execute_transactions(info_connect)
    
    # Information Units and Subject Relation
    triplet_relation = []
    for i in range(len(df_temp)):
        info = df_temp.iloc[i,0]
        sub = df_temp.iloc[i,1]
        triplet_relation.append(f'''MATCH (a:info_unit), (b:subject_object) 
                        WHERE a.info_unit = "{info}" AND b.sub_obj = "{sub}"
                        create (a) -[r:contain]-> (b)''')
    execute_transactions(triplet_relation)

    # Triplets    
    triplets = []
    for i in range(len(df_temp)):
        sub = df_temp.iloc[i,1]
        pred = df_temp.iloc[i,2]
        obj = df_temp.iloc[i,3]
        triplets.append(f'''MATCH (a:subject_object), (b:subject_object) 
                        WHERE a.sub_obj = "{sub}" AND b.sub_obj = "{obj}"
                        create (a) -[r: {pred}]-> (b)''')
    execute_transactions(triplets)  
    
    df_temp = df_research[df_research['paper_ID'] == paper_id]
    research_connect = []
    for i in range(len(df_temp)):
        obj = df_temp.iloc[i,-1]
        research_connect.append(f'''MATCH (a:paper_id), (b:subject_object) 
                                WHERE a.paper_id = "{paper}" AND b.info_unit = "{obj}"
                                create (a) -[r:has_research_problem]-> (b)''')
    execute_transactions(research_connect)

In [ ]:
for i in range(0,10):
    print(i)
    paper_graph('entailment', i)

In [ ]:
info_units = set()
subject_object = set()

In [ ]:
def paper_graph(topic, paper_id):
    
    df_temp = df[(df['topic'] == topic) & (df['paper_ID'] == paper_id)]
    df_temp = df_temp[['info-unit', 'sub', 'pred', 'obj']]
    
    paper = str(paper_id)
    
    paper_node = ['create (t:paper {paper_id : "' + paper + '"})']
    execute_transactions(paper_node)
    
    info_unit = df_temp['info-unit'].unique()
    sub_obj = pd.concat([df_temp['sub'], df_temp['obj']]).unique()
    
    
    # Information Units Nodes
    info_nodes = []
    info_connect = []
    for i, info in enumerate(info_unit):
        if info not in info_units:
            info_nodes.append('create (t:info_unit {info_unit : "' + info + '"})')
            info_connect.append(f'''MATCH (a:paper_id), (b:info_unit) 
                                WHERE a.paper_id = "{paper}" AND b.info_unit = "{info}"
                                create (a) -[r:has]-> (b)''')
    execute_transactions(info_nodes)
    execute_transactions(info_connect)
    
    # Subject/Object Nodes
    subject_nodes = []
    for i, sub in enumerate(sub_obj):
        if sub not in subject_object:
            subject_nodes.append('create (t:subject_object {sub_obj: "' + sub + '"})')
    execute_transactions(subject_nodes)
    
    # Relations
    relation = []
    for i in range(len(df_temp)):
        info = df_temp.iloc[i,0]
        sub = df_temp.iloc[i,1]
        relation.append(f'''MATCH (a:info_unit), (b:subject_object) 
                        WHERE a.info_unit = "{info}" AND b.sub_obj = "{sub}"
                        create (a) -[r:contain]-> (b)''')
    execute_transactions(relation)

    # Triplets    
    triplets = []
    for i in range(len(df_temp)):
        pred = df_temp.iloc[i,2]
        pred = pred.replace(" ", "_")
        sub = df_temp.iloc[i,1]
        obj = df_temp.iloc[i,3]
        triplets.append(f'''MATCH (a:subject_object), (b:subject_object) 
                        WHERE a.sub_obj = "{sub}" AND b.sub_obj = "{obj}"
                        create (a) -[r: {pred}]-> (b)''')
    execute_transactions(triplets)    
    
    df_temp = df_research[df_research['paper_ID'] == paper_id]
    research_nodes = []
    research_connect = []
    for i in range(len(df_temp)):
        obj = df_temp.iloc[i,-1]
        if obj not in subject_object:
            research_nodes.append('create (t:subject_object {sub_obj: "' + obj + '"})')
            research_connect.append(f'''MATCH (a:paper_id), (b:subject_object) 
                                WHERE a.paper_id = "{paper}" AND b.info_unit = "{obj}"
                                create (a) -[r:has_research_problem]-> (b)''')
    execute_transactions(research_nodes)
    execute_transactions(research_connect)